In [21]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)
from pathlib import Path
import csv


from datetime import date, datetime

**EDA**

In [3]:
df = pd.read_csv('Camp_Market.csv', sep=';'))
df.head()

#lis le fichier/charge les données


SyntaxError: unmatched ')' (2173524068.py, line 1)

In [ ]:
df.shape
#affiche le nb de ligne et de colonne

In [ ]:
df.info()

In [ ]:
df.isnull().sum()
#affiche le nb de valeur manquantes/nulles

In [ ]:
#conversion des colonnes to DateTime format
df["Dt_Customer"] = pd.to_datetime(df["Dt_Customer"], dayfirst=True, errors="coerce")

df.rename(columns = {'MntGoldProds':'MntGoldProducts'}, inplace = True)


In [ ]:
#clients sans revenue (revenus manquants)
clients_sans_income = df[df['Income'].isna()]
clients_sans_income.head()
clients_sans_income.shape[0]

**Feature engineering**

In [ ]:
#tri le nb de valeurs pour "education"
df['Education'].value_counts()

In [ ]:
df['Marital_Status'].value_counts()

In [ ]:
#drop les colonnes inutiles
df.drop(['ID','Z_CostContact','Z_Revenue'], axis=1, inplace=True)

In [ ]:
df["Education_Level"]=df["Education"].replace({"Basic":"Undergraduate","2n Cycle":"Undergraduate", "Graduation":"Graduate", "Master":"Postgraduate", "PhD":"Postgraduate"})

In [ ]:
#quantité total depensée
df["TotalMntSpent"] = df["MntWines"] + df["MntFruits"] + df["MntMeatProducts"] + df["MntFishProducts"] + df["MntSweetProducts"] + df["MntGoldProducts"]

#statut marital parternairfe ou non
df["Partner"] = df["Marital_Status"].replace({"Married": "Yes", "Together": "Yes", "Absurd": "No", "Widow": "No", "YOLO": "No", "Divorced": "No", "Single": "No", "Alone": "No"})

#segmentation du niveau d'education
df["Education_Level"]=df["Education"].replace({"Basic":"Undergraduate","2n Cycle":"Undergraduate", "Graduation":"Graduate", "Master":"Postgraduate", "PhD":"Postgraduate"})

#calcul total d'enfants
df["Children"] = df["Kidhome"] +  df["Teenhome"]

df['Years_Customer'] = (df["Dt_Customer"].dt.year.max()) - (df['Dt_Customer'].dt.year)
df['Days_Customer'] = (df["Dt_Customer"].max()) - (df['Dt_Customer'])

#calcul age
df["Age"] = (df["Dt_Customer"].dt.year.max()) - (df['Year_Birth'].dt.year)

#division des ages en groupe
df['Age_Group'] = pd.cut(x = df['Age'], bins = [1, 17, 24, 44, 64, 150], labels = ['Under 18','Young adult','Adult','Middel Aged','Senior'])

#nb total de campagnes acceptées
df['Total_Acc_Cmp'] = df['AcceptedCmp1'] + df['AcceptedCmp2'] + df['AcceptedCmp3'] + df['AcceptedCmp4'] + df['AcceptedCmp5'] + df['Response']




In [ ]:
print(df.shape)
df.head()

In [ ]:
df["Dt_Customer"] = pd.to_datetime(df["Dt_Customer"], dayfirst=True, errors="coerce")


In [18]:
#valeurs uniques dans le dataset selon chaque colonnes
unique_number = []
for i in df.columns:
    x = df[i].value_counts().count()
    unique_number.append(x)

pd.DataFrame(unique_number, index = df.columns, columns = ["Total Unique Values"])

NameError: name 'df' is not defined

In [ ]:
#Graphique histogramme sur variables numériques
num_cols = df.select_dtypes(include=['int', 'float']).columns

# Nombre de colonnes numériques
n = len(num_cols)

# Dimensions de la grille (ajuste selon ton nombre de colonnes)
n_cols = 3
n_rows = (n + n_cols - 1) // n_cols  # arrondi vers le haut

fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5 * n_rows))
axes = axes.flatten()  # pour parcourir facilement les axes
colors = ['#8A2BE2', '#7B68EE', '#6A5ACD', '#483D8B', '#4169E1', '#6495ED']  # 6 tons

for i, col in enumerate(num_cols):
    color = colors[i % len(colors)]
    df[col].plot(kind='hist',bins=20, color=color, edgecolor='black', ax=axes[i])

# Masquer les cases vides si le nombre de colonnes n’est pas multiple de n_cols
for j in range(i + 1, len(axes)):
    axes[j].set_visible(False)

plt.tight_layout()
plt.show()


In [2]:
# Calcul de la matrice de corrélation
corr_matrix = df.corr(numeric_only=True)

# Taille du graphique
plt.figure(figsize=(14, 10))

# Heatmap
sns.heatmap(
    corr_matrix,
    annot=True,
    fmt=".2f",
    cmap="coolwarm",
    center=0,
    square=True,
    linewidths=0.5,
    cbar_kws={"shrink": 0.8},
    annot_kws={"size": 8})

plt.title("Matrice de corrélation", fontsize=16, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

NameError: name 'df' is not defined

In [ ]:
# Normalisation des colonnes numériques
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# sélectionner les colonnes numériques
num_cols = df.select_dtypes(include=['int', 'float']).columns.tolist()
print('Colonnes numériques trouvées :', num_cols)

# imputer les valeurs manquantes avec la médiane (sécurisé pour outliers)
imputer = SimpleImputer(strategy='median')
df_num = pd.DataFrame(imputer.fit_transform(df[num_cols]), columns=num_cols, index=df.index)

# Standardisation (moyenne 0, écart-type 1)
scaler = StandardScaler()
df_std = pd.DataFrame(scaler.fit_transform(df_num), columns=[f'{c}_std' for c in num_cols], index=df.index)

# Min-Max scaling (0-1)
mmsc = MinMaxScaler()
df_mm = pd.DataFrame(mmsc.fit_transform(df_num), columns=[f'{c}_mm' for c in num_cols], index=df.index)

# concaténer les colonnes normalisées au DataFrame original (optionnel : assigner en place)
df = pd.concat([df, df_std, df_mm], axis=1)

print('Normalisation terminée. Colonnes ajoutées :', list(df_std.columns)[:5], '...')
